In [19]:
!pip install -q -U kaggle_environments

In [20]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Агент 1. Всегда играет "камень" - это значение 0

In [ ]:
%%writefile rock_agent.py

#Всегда выбирает 0 - камень
def rock_agent(observation, configuration):
    return 0

Writing rock_agent.py


Агент 2. Всегда играет "бумага" - это значение 1

In [21]:
%%writefile paper_agent.py

#Всегда выбирает 1 - бумага
def paper_agent(observation, configuration):
    return 0

Overwriting paper_agent.py


Агент 3. Всегда играет "ножницы" - это значение 2

In [22]:
%%writefile scissors_agent.py

#Всегда выбирает 2 - ножницы
def scissors_agent(observation, configuration):
    return 0

Overwriting scissors_agent.py


Агент 4. Всегда выбирает случайное действие.

In [23]:
%%writefile random_agent.py

def random_agent(observation, configuration):
    return random.randrange(0, configuration.signs)


Overwriting random_agent.py


Агент 5. Производит то же самое действие, что и оппонент на прошлом ходу. На первом ходу - случаный выбор.

In [24]:
%%writefile copy_opponent.py

def copy_opponent(observation, configuration):
    #Если первый ход то идем рандомно
    if observation.step == 0:
        return random.randrange(0, configuration.signs)
    #На 2 и остальных ходах выбираем то что ход назад выбирал противник
    else:
        return observation.lastOpponentAction


Overwriting copy_opponent.py


Агент 6. Производит случайно действия кроме того, что оппонент на прошлом ходу. На первом ходу - случаный выбор.

In [25]:
%%writefile not_copy_opponent.py

def not_copy_opponent(observation, configuration):
    #Если первый ход то идем рандомно
    if observation.step == 0:
        return random.randrange(0, configuration.signs)
    #На 2 и остальных ходах выбираем НЕ то что ход назад выбирал противник
    else:
        numbers = list(range(0, configuration.signs))
        # удаляем из списка действий то, которое призвел противник на прошлом ходу и случаной выбираем между оставшимся
        newnambers = numbers.remove(observation.lastOpponentAction)
        not_copy_opponent = random.choice(newnambers)
        return not_copy_opponent

Overwriting not_copy_opponent.py


Агент 7. Производит действие, которое бы било прошлый выбор противника. На первом ходу - случаный выбор.

In [26]:
%%writefile react_action.py

last_react_action = None
def reactionary(observation, configuration):
    global last_react_action
    # Если это первый шаг, задаем случайное действие
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    # В остальных случаях, выбираем то что било бы прошлый выбор противника.
    else:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_react_action

Overwriting react_action.py


Агент 8. Не производит действие, которое бы било прошлый выбор противника. На первом ходу - случаный выбор.

In [27]:
%%writefile unreact_action.py

last_react_action = None
def unreactionary(observation, configuration):
    global last_react_action
    # Если это первый шаг, задаем случайное действие
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    # В остальных случаях, выбираем то что НЕ било бы прошлый выбор противника.
    else:
        numbers = list(range(0, configuration.signs))
        # удаляем из списка действий то, которое било бы противника на прошлом ходу и случаной выбираем между оставшимся
        newnambers = numbers.remove(observation.lastOpponentAction)
        last_react_action = random.choice(newnambers)
    return last_react_action

Overwriting unreact_action.py


Агент 9. Производит действие, которое:
*   если противник выиграл то выбирем то что мы били бы в прошлый раз
*   В остальных случаях выбираем то что било бы противника в прошлый раз

На первом ходу - случаный выбор.

In [28]:
%%writefile counter_reaction.py

last_counter_action = None

def counter_reactionary(observation, configuration):
    global last_counter_action
    # Если это первый шаг, задаем случайное действие
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)
    # Узнаем, выиграл ли противник в прошлый раз
    elif get_score(last_counter_action, observation.lastOpponentAction) == 1:
        # Если противник выиграл то выбирем то что мы били бы в прошлый раз
        last_counter_action = (last_counter_action + 2) % configuration.signs
    # В остальных случаях выбираем то что било бы противника в прошлый раз
    else:
        last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_counter_action

Overwriting counter_reaction.py


Агент 10. Производит действие, которое:
*   если противник выиграл то выбираем то что било бы противника в прошлый раз
*   В остальных случаях выбирем то что мы били бы в прошлый раз

На первом ходу - случаный выбор.

In [29]:
%%writefile uncounter_reaction.py

last_uncounter_action = None

def uncounter_reactionary(observation, configuration):
    global last_uncounter_action
    # Если это первый шаг, задаем случайное действие
    if observation.step == 0:
        last_uncounter_action = random.randrange(0, configuration.signs)
    # Узнаем, выиграл ли противник в прошлый раз
    elif get_score(last_uncounter_action, observation.lastOpponentAction) == 1:
        last_uncounter_action = (observation.lastOpponentAction + 1) % configuration.signs
    else:
        last_uncounter_action = (last_uncounter_action + 2) % configuration.signs

    return last_uncounter_action

Writing uncounter_reaction.py


Агент 11. Повторяет свой прошлый выбор, если в прошлый раз выиграл, если нет - рандом. На первом ходу - случаный выбор.

In [30]:
%%writefile repeat_if_win.py

last_react_action = None
def repeat_if_win(observation, configuration):
    global last_react_action
    # Если это первый шаг, задаем случайное действие и возвращаем его
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
        return last_react_action
    # Проверяем результат предыдущего действия
    if get_score(last_react_action, observation.lastOpponentAction) > 0:
        # Если выиграли, повторяем то же действие
        return last_react_action
    else:
        # Если проиграли или ничья, выбираем новое случайное действие
        last_react_action = random.randrange(0, configuration.signs)
        return last_react_action

Writing repeat_if_win.py


Агент 12. Повторяет свой прошлый выбор, если в прошлый раз проиграл, если нет - рандом. На первом ходу - случаный выбор.

In [31]:
%%writefile repeat_if_lose.py

last_react_action = None

def repeat_if_lose(observation, configuration):
    global last_react_action
    # Если это первый шаг, задаем случайное действие и возвращаем его
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
        return last_react_action
    # Проверяем результат предыдущего действия
    if get_score(last_react_action, observation.lastOpponentAction) < 0:
        # Если проиграли, повторяем то же действие
        return last_react_action
    else:
        # Если выиграли или ничья, выбираем новое случайное действие
        last_react_action = random.randrange(0, configuration.signs)
        return last_react_action

Writing repeat_if_lose.py


Агент 13. Перебирает на первом ходу камень, на втором - бумага, на третьем - ножницы, на четветом - камень,  и так - по кругу

In [32]:
%%writefile zero_one_two.py

def zero_one_two(observation, configuration):
        return (observation.step + 2) % configuration.signs

Writing zero_one_two.py


Агент 14.

Перебирает на первом ходу камень, на втором - бумага, на третьем - ножницы, на четветом - камень,  и так - по кругу.

Когда выигрывает - повторяет что было в прошлый раз.

In [33]:
%%writefile repeat_if_win_or_zero_one_two.py

last_react_action = 0
def repeat_if_win_or_zero_one_two(observation, configuration):
    global last_react_action
    # Для не первого шага:
    if observation.step > 0:
        # Проверяем результат предыдущего действия
        if get_score(last_react_action, observation.lastOpponentAction) > 0:
           # Если выиграли, повторяем то же действие
           return last_react_action
        else:
            # Если проиграли или ничья, выбираем новое случайное действие
            last_react_action = (observation.step + 2) % configuration.signs
    return last_react_action

Writing repeat_if_win_or_zero_one_two.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["random_agent.py", "scissors_agent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "scissors_agent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[0, 0]]